In [15]:
import pyomo.environ as pyo

In [16]:
model = pyo.ConcreteModel()

# 변수정의
model.prod = pyo.Set(initialize=['PE','PP'])
model.prec = pyo.Set(initialize=['ethylene','propylene'])
model.profit = pyo.Param(model.prod, initialize={'PE':3,'PP':2})
model.raw = pyo.Param(model.prod, model.prec, initialize={('PE','ethylene'):1,('PE','propylene'):0,('PP','ethylene'):0,('PP','propylene'):1})
model.energy = pyo.Param(model.prod, initialize={'PE':2,'PP':1})

model.x = pyo.Var(model.prod, within=pyo.NonNegativeReals)

# 목적함수
model.obj = pyo.Objective(expr=sum(model.profit[i]*model.x[i] for i in model.prod), sense=pyo.maximize)

# 제약조건
model.con_raw = pyo.Constraint(expr=sum(model.raw[i, j]*model.x[i] for i in model.prod for j in model.prec) <= 80)
model.con_energy = pyo.Constraint(expr=sum(model.energy[i]*model.x[i] for i in model.prod) <= 100)
model.con_PE = pyo.Constraint(expr=model.x['PE'] <= 40)

model.solver = pyo.SolverFactory('ipopt')

# 결과 출력
# pyomo 계산 결과를 출력하는 코드
result = model.solver.solve(model)
for f in model.prod:
    print(f"{f}: {pyo.value(model.x[f])}")

print(f"\n목적함수 값: {pyo.value(model.obj)}")

# 각 제약 조건이 어떻게 만족되었는지 출력
print("\n각 제약 조건의 실제 값:")
print(f"총 원료 사용량: {sum(pyo.value(model.raw[i, j]) * pyo.value(model.x[i]) for i in model.prod for j in model.prec)} (<= 80)")
print(f"총 에너지 사용량: {sum(pyo.value(model.energy[i]) * pyo.value(model.x[i]) for i in model.prod)} (<= 100)")
print(f"PE 생산량: {pyo.value(model.x['PE'])} (>= 40)")


PE: 20.000000200000173
PP: 60.00000059749381

목적함수 값: 180.00000179498812

각 제약 조건의 실제 값:
총 원료 사용량: 80.00000079749398 (<= 80)
총 에너지 사용량: 100.00000099749415 (<= 100)
PE 생산량: 20.000000200000173 (>= 40)
